# Electricity Markets

:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).
Then install the following packages by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install -q pypsa highspy
```
:::

## What is PyPSA?

PyPSA stands for Python for Power System Analysis. The [documentation](https://pypsa--1250.org.readthedocs.build/en/1250/) describes it as follows:

> PyPSA is an open-source Python framework for optimising and simulating modern power and energy systems that include features such as conventional generators with unit commitment, variable wind and solar generation, hydro-electricity, inter-temporal storage, coupling to other energy sectors, elastic demands, and linearised power flow with loss approximations in DC and AC networks. PyPSA is designed to scale well with large networks and long time series. It is made for researchers, planners and utilities with basic coding aptitude who need a fast, easy-to-use and transparent tool for power and energy system analysis.

For us, it's a tool to model electricity markets and investment planning as optimisation problems as discussed in the slides.

PyPSA is a way to describe energy systems in Python code, interface with solver software, and analyse and visualise the results.

:::{note}
For a full reference to the optimisation problem description, see the [PyPSA User Guide](https://pypsa--1250.org.readthedocs.build/en/1250/user-guide/optimization/overview/)
:::

## Input Data

Let's dive straight in and get acquainted with PyPSA by building simple electricity market models.

We have the following data:

- fuel costs in € / MWh$_{th}$ 

In [1]:
fuel_cost = dict(
    coal=8,
    gas=100,
    oil=48,
    hydro=1,
    wind=0,
)

- efficiencies of thermal power plants in MWh$_{el}$ / MWh$_{th}$

In [2]:
efficiency = dict(
    coal=0.33,
    gas=0.58,
    oil=0.35,
    hydro=1,
    wind=1,
)

- specific emissions in t$_{CO_2}$ / MWh$_{th}$

In [3]:
# t/MWh thermal
emissions = dict(
    coal=0.34,
    gas=0.2,
    oil=0.26,
    hydro=0,
    wind=0,
)

- power plant capacities in MW

In [4]:
power_plants = {
    "A": {"coal": 35000, "wind": 3000, "gas": 8000, "oil": 2000},
    "B": {"hydro": 1200},
}

- fixed electrical load in MW

In [5]:
loads = {
    "A": 42000,
    "B": 650,
}

## Building the Network

By convention, PyPSA is imported without an alias:

In [6]:
import pypsa

First, we create a new network object which serves as the overall container for all components.

In [7]:
n = pypsa.Network()

The second component we need are buses. **Buses** are the fundamental nodes of the network, to which all other components like loads, generators and transmission lines attach. They enforce energy conservation for all elements feeding in and out of it.

Components can be added to the network `n` using the `n.add()` function. It takes the component name as a first argument, the name of the component as a second argument and possibly further parameters as keyword arguments. Let's use this function, to add buses for each country to our network:

In [9]:
n.add("Bus", "A", v_nom=380, carrier="AC")
n.add("Bus", "B", v_nom=380, carrier="AC");

For each class of components, the data describing the components is stored in a `pandas.DataFrame`. For example, all static data for buses is stored in `n.buses`

In [10]:
n.buses

,v_nom,type,x,y,carrier,unit,location,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,generator,sub_network
name,,,,,,,,,,,,,
A,380.0,,0.0,0.0,AC,,,1.0,0.0,inf,PQ,,
B,380.0,,0.0,0.0,AC,,,1.0,0.0,inf,PQ,,


You see there are many more attributes than we specified while adding the buses; many of them are filled with default parameters which were added. You can look up the field description, defaults and status (required input, optional input, output) for buses here https://pypsa--1250.org.readthedocs.build/en/1250/user-guide/components/buses/, and analogous for all other components. 

The method `n.add()` also allows you to add multiple components at once. For instance, multiple **carriers** for the fuels with information on specific carbon dioxide emissions, a nice name, and colors for plotting. For this, the function takes the component name as the first argument and then a list of component names and then optional arguments for the parameters. Here, scalar values, lists, dictionary or `pandas.Series` are allowed. The latter two needs keys or indices with the component names.

In [11]:
n.add(
    "Carrier",
    ["coal", "gas", "oil", "hydro", "wind"],
    co2_emissions=emissions,
    nice_name=["Coal", "Gas", "Oil", "Hydro", "Onshore Wind"],
    color=["grey", "indianred", "black", "aquamarine", "dodgerblue"],
);

The `n.add()` function is very general. It lets you add any component to the network object `n`. For instance, in the next step we add **generators** for all the different power plants.

In Region B:

In [12]:
n.add(
    "Generator",
    "B hydro",
    bus="B",
    carrier="hydro",
    p_nom=1200,  # MW
);

In Region A (in a loop):

In [13]:
for tech, p_nom in power_plants["A"].items():
    n.add(
        "Generator",
        f"A {tech}",
        bus="A",
        carrier=tech,
        efficiency=efficiency.get(tech, 1),
        p_nom=p_nom,
        marginal_cost=fuel_cost.get(tech, 0) / efficiency.get(tech, 1),
    )

As a result, the `n.generators` DataFrame looks like this:

In [14]:
n.generators["marginal_cost"]

name
B hydro      0.000000
A coal      24.242424
A wind       0.000000
A gas      172.413793
A oil      137.142857
Name: marginal_cost, dtype: float64

Next, we're going to add the electricity demand.

A positive value for `p_set` means consumption of power from the bus.

In [15]:
n.add(
    "Load",
    "A electricity demand",
    bus="A",
    p_set=loads["A"],
);

In [16]:
n.add(
    "Load",
    "B electricity demand",
    bus="B",
    p_set=loads["B"],
);

In [17]:
n.loads

,bus,carrier,type,p_set,q_set,sign,active
name,,,,,,,
A electricity demand,A,,,42000.0,0.0,-1.0,True
B electricity demand,B,,,650.0,0.0,-1.0,True


Finally, we add the connection between Region B and Region A with a 500 MW line:

In [18]:
n.add(
    "Line",
    "A-B",
    bus0="A",
    bus1="B",
    s_nom=500,
    x=1,
    r=1,
);

In [19]:
n.lines

,bus0,bus1,type,x,r,g,b,s_nom,s_nom_mod,s_nom_extendable,...,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
name,,,,,,,,,,,,,,,,,,,,,
A-B,A,B,,1.0,1.0,0.0,0.0,500.0,0.0,False,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Optimisation

With all input data transferred into PyPSA's data structure, we can now build and run the resulting optimisation problem. In PyPSA, building, solving and retrieving results from the optimisation model is contained in a single function call `n.optimize()`. This function optimizes dispatch and investment decisions for least cost.

The `n.optimize()` function can take a variety of arguments. The most relevant for the moment is the choice of the solver (e.g. "highs" and "gurobi"). They need to be installed on your computer, to use them here!

In [20]:
n.optimize(log_to_console=False)

Index(['A', 'B'], dtype='object', name='name')
Index(['A-B'], dtype='object', name='name')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.model:Solver options:
 - log_to_console: False
INFO:linopy.io: Writing time: 0.01s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 6 primals, 14 duals
Objective: 1.38e+06
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Line-fix-s-lower, Line-fix-s-upper were not assigned to the network.


Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms


('ok', 'optimal')

Let's have a look at the results.

Since the power flow and dispatch are generally time-varying quantities, these are stored in a different location than e.g. `n.generators`. They are stored in `n.generators_t`. Thus, to find out the dispatch of the generators, run

In [21]:
n.generators_t.p

name,B hydro,A coal,A wind,A gas,A oil
snapshot,,,,,
now,1150.0,35000.0,3000.0,1500.0,2000.0


or if you prefer it in relation to the generators nominal capacity

In [22]:
n.generators_t.p / n.generators.p_nom

name,B hydro,A coal,A wind,A gas,A oil
snapshot,,,,,
now,0.958333,1.0,1.0,0.1875,1.0


You see that the time index has the value 'now'. This is the default index when no time series data has been specified and the network only covers a single state (e.g. a particular hour). 

Similarly you will find the power flow in transmission lines at

In [23]:
n.lines_t.p0

name,A-B
snapshot,
now,-500.0


In [24]:
n.lines_t.p1

name,A-B
snapshot,
now,500.0


The `p0` will tell you the flow from `bus0` to `bus1`. `p1` will tell you the flow from `bus1` to `bus0`.

What about the _market clearing prices_ in the electricity market?

In [25]:
n.buses_t.marginal_price

name,A,B
snapshot,,
now,172.413793,-0.0


## Modifying networks

Modifying data of components in an existing PyPSA network can be done by modifying the entries of a `pandas.DataFrame`. For instance, if we want to reduce the cross-border transmission capacity between Region A and Region B, we'd run:

In [26]:
n.lines.loc["A-B", "s_nom"] = 400

In [27]:
n.lines

,bus0,bus1,type,x,r,g,b,s_nom,s_nom_mod,s_nom_extendable,...,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt,v_nom
name,,,,,,,,,,,,,,,,,,,,,
A-B,A,B,,1.0,1.0,0.0,0.0,400.0,0.0,False,...,inf,0,0.000007,0.000007,0.0,0.0,0.000007,0.000007,500.0,380.0


In [28]:
n.optimize(solver_name="highs")

Index(['0'], dtype='object', name='name')
Index(['A', 'B'], dtype='object', name='name')
Index(['A-B'], dtype='object', name='name')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.01s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 6 primals, 14 duals
Objective: 1.40e+06
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Line-fix-s-lower, Line-fix-s-upper were not assigned to the network.


Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
LP   linopy-problem-fdmy6yxt has 14 rows; 6 cols; 19 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [2e+01, 2e+02]
  Bound  [0e+00, 0e+00]
  RHS    [4e+02, 4e+04]
Presolving model
1 rows, 2 cols, 2 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-14); columns 0(-6); elements 0(-19) - Reduced to empty
Solving the original LP from the solution after postsolve
Model name          : linopy-problem-fdmy6yxt
Model status        : Optimal
Objective value     :  1.3986326317e+06
P-D objective error :  1.6647013314e-16
HiGHS run time      :          0.00
Writing the solution to /tmp/linopy-solve-h2oo98_s.sol


('ok', 'optimal')

You can see that the production of the hydro power plant was reduced and that of the gas power plant increased owing to the reduced transmission capacity.

In [29]:
n.generators_t.p

name,B hydro,A coal,A wind,A gas,A oil
snapshot,,,,,
now,1050.0,35000.0,3000.0,1600.0,2000.0


## Exploring the optimization model

In [30]:
n.model

Linopy LP model

Variables:
----------
 * Generator-p (snapshot, name)
 * Line-s (snapshot, name)

Constraints:
------------
 * Generator-fix-p-lower (snapshot, name)
 * Generator-fix-p-upper (snapshot, name)
 * Line-fix-s-lower (snapshot, name)
 * Line-fix-s-upper (snapshot, name)
 * Bus-nodal_balance (name, snapshot)

Status:
-------
ok

In [31]:
n.model.constraints["Generator-fix-p-upper"]

Constraint `Generator-fix-p-upper` [snapshot: 1, name: 5]:
----------------------------------------------------------
[now, B hydro]: +1 Generator-p[now, B hydro] ≤ 1200.0
[now, A coal]: +1 Generator-p[now, A coal]   ≤ 35000.0
[now, A wind]: +1 Generator-p[now, A wind]   ≤ 3000.0
[now, A gas]: +1 Generator-p[now, A gas]     ≤ 8000.0
[now, A oil]: +1 Generator-p[now, A oil]     ≤ 2000.0

In [32]:
n.model.constraints["Bus-nodal_balance"]

Constraint `Bus-nodal_balance` [name: 2, snapshot: 1]:
------------------------------------------------------
[A, now]: +1 Generator-p[now, A coal] + 1 Generator-p[now, A wind] + 1 Generator-p[now, A gas] + 1 Generator-p[now, A oil] - 1 Line-s[now, A-B] = 42000.0
[B, now]: +1 Generator-p[now, B hydro] + 1 Line-s[now, A-B]                                                                                     = 650.0

In [33]:
n.model.objective

Objective:
----------
LinearExpression: +0 Generator-p[now, B hydro] + 24.24 Generator-p[now, A coal] + 0 Generator-p[now, A wind] + 172.4 Generator-p[now, A gas] + 137.1 Generator-p[now, A oil]
Sense: min
Value: 1398632.6317348

## Exercises

**Task 1:** Model an outage of the transmission line by setting it's capacity to zero. How does the model compensate for the lack of transmission? 

In [ ]:
n.lines["s_nom"] = 0.0
n.optimize(log_to_console=False)
n.generators_t.p

**Task 2:** Double the wind capacity. How is the electricity price affected? What generator is price-setting?

In [ ]:
n.generators.loc["A wind", "p_nom"] *= 2
n.optimize(log_to_console=False)
n.buses_t.marginal_price

**Task 3:** Region A brings a new 1000 MW nuclear power plant into operation with an estimated marginal electricity generation cost of 20 €/MWh. Will that affect the electricity price?

In [ ]:
n.add(
    "Generator",
    "A nuclear",
    bus="A",
    carrier="nuclear",
    p_nom=1000,
    marginal_cost=20,
)
n.optimize(log_to_console=False)
n.buses_t.marginal_price